This code will filter the surfaces with a negative formation energy. 
Key values are saved into a data base file called filtered_surfaces.db
Next steps: adsorption site finder and adsorption energy calculations 

In [1]:
import sqlite3
import json
import os

# Configuration
input_db = 'surface_formation_energies_uma.db'
output_db = 'filtered_surfaces.db'
energy_threshold = 0.0

# Delete output if exists
if os.path.exists(output_db):
    os.remove(output_db)

# Read source database
source_conn = sqlite3.connect(input_db)
source_cursor = source_conn.cursor()

# Get all rows
source_cursor.execute("SELECT * FROM systems")
all_rows = source_cursor.fetchall()
source_conn.close()

# Index of key_value_pairs
kvp_index = 25   # from schema inspection

# Filter rows by energy
kept_rows = []
kept_count = 0
removed_count = 0

for row in all_rows:
    kvp_text = row[kvp_index]

    if kvp_text is None:
        removed_count += 1
        continue

    try:
        if isinstance(kvp_text, bytes):
            kvp_text = kvp_text.decode('utf-8')

        data = json.loads(kvp_text)
        gamma = data.get('surface_formation_energy_ev_per_ang2')

        if gamma is not None and gamma <= energy_threshold:
            kept_rows.append(row)
            kept_count += 1
        else:
            removed_count += 1
    except Exception:
        removed_count += 1

# Create new database
new_conn = sqlite3.connect(output_db)
new_cursor = new_conn.cursor()

# Copy original schema
orig_conn = sqlite3.connect(input_db)
orig_cursor = orig_conn.cursor()
orig_cursor.execute("SELECT sql FROM sqlite_master WHERE name='systems' AND type='table'")
result = orig_cursor.fetchone()
orig_conn.close()

create_table_sql = result[0]  # FIXED: extract string

new_cursor.execute(create_table_sql)

# Insert filtered records
placeholders = ", ".join("?" * len(all_rows[0]))
insert_sql = f"INSERT INTO systems VALUES ({placeholders})"

for row in kept_rows:
    new_cursor.execute(insert_sql, row)

new_conn.commit()
new_conn.close()

print(f"Total entries: {len(all_rows)}")
print(f"Kept (γ ≤ {energy_threshold}): {kept_count}")
print(f"Removed: {removed_count}")
print(f"\n✓ Saved {kept_count} structures to {output_db}")


Total entries: 29335
Kept (γ ≤ 0.0): 17723
Removed: 11612

✓ Saved 17723 structures to filtered_surfaces.db


Now that the data base is filtered by surface formation energy, lets see what we stored inside it

In [4]:
conn = sqlite3.connect('surface_formation_energies_uma.db')
cur = conn.cursor()

cur.execute("PRAGMA table_info(systems)")
for col in cur.fetchall():
    print(col)

conn.close()


(0, 'id', 'INTEGER', 0, None, 1)
(1, 'unique_id', 'TEXT', 0, None, 0)
(2, 'ctime', 'REAL', 0, None, 0)
(3, 'mtime', 'REAL', 0, None, 0)
(4, 'username', 'TEXT', 0, None, 0)
(5, 'numbers', 'BLOB', 0, None, 0)
(6, 'positions', 'BLOB', 0, None, 0)
(7, 'cell', 'BLOB', 0, None, 0)
(8, 'pbc', 'INTEGER', 0, None, 0)
(9, 'initial_magmoms', 'BLOB', 0, None, 0)
(10, 'initial_charges', 'BLOB', 0, None, 0)
(11, 'masses', 'BLOB', 0, None, 0)
(12, 'tags', 'BLOB', 0, None, 0)
(13, 'momenta', 'BLOB', 0, None, 0)
(14, 'constraints', 'TEXT', 0, None, 0)
(15, 'calculator', 'TEXT', 0, None, 0)
(16, 'calculator_parameters', 'TEXT', 0, None, 0)
(17, 'energy', 'REAL', 0, None, 0)
(18, 'free_energy', 'REAL', 0, None, 0)
(19, 'forces', 'BLOB', 0, None, 0)
(20, 'stress', 'BLOB', 0, None, 0)
(21, 'dipole', 'BLOB', 0, None, 0)
(22, 'magmoms', 'BLOB', 0, None, 0)
(23, 'magmom', 'REAL', 0, None, 0)
(24, 'charges', 'BLOB', 0, None, 0)
(25, 'key_value_pairs', 'TEXT', 0, None, 0)
(26, 'data', 'BLOB', 0, None, 0)
(27, '

In [5]:
import sqlite3, json

conn = sqlite3.connect('surface_formation_energies_uma.db')
cur = conn.cursor()

cur.execute("SELECT key_value_pairs FROM systems LIMIT 3")
for row in cur.fetchall():
    kvp = row[0]
    if isinstance(kvp, bytes):
        kvp = kvp.decode('utf-8')
    data = json.loads(kvp)
    print(json.dumps(data, indent=2))
    print("-" * 40)

conn.close()


{
  "structure_idx": 0,
  "miller_index": "(1, 0, 0)",
  "termination_index": 0,
  "composition": "{\"Zr\": 20, \"Os\": 12}",
  "n_atoms": 32,
  "n_layers": 8,
  "n_fixed_atoms": 8,
  "slab_area_ang2": 46.28384989871256,
  "bulk_reference_energy_ev": -288.514652,
  "total_slab_energy_ev": -286.81938100663126,
  "surface_formation_energy_ev_per_ang2": 0.01831385026395468,
  "calculation_date": "2025-12-03T17:13:35.710606"
}
----------------------------------------
{
  "structure_idx": 0,
  "miller_index": "(1, 0, 0)",
  "termination_index": 1,
  "composition": "{\"Zr\": 20, \"Os\": 12}",
  "n_atoms": 32,
  "n_layers": 9,
  "n_fixed_atoms": 7,
  "slab_area_ang2": 46.28384989871256,
  "bulk_reference_energy_ev": -288.514652,
  "total_slab_energy_ev": -286.9992096504301,
  "surface_formation_energy_ev_per_ang2": 0.016371178638837445,
  "calculation_date": "2025-12-03T17:13:35.820686"
}
----------------------------------------
{
  "structure_idx": 0,
  "miller_index": "(1, 0, 0)",
  "termin

In [6]:
import sqlite3, json

conn = sqlite3.connect('filtered_surfaces.db')
cur = conn.cursor()

cur.execute("SELECT id, key_value_pairs FROM systems LIMIT 5")
for row in cur.fetchall():
    kvp = row[1]
    if isinstance(kvp, bytes):
        kvp = kvp.decode('utf-8')

    data = json.loads(kvp)
    gamma = data.get('surface_formation_energy_ev_per_ang2')
    
    # Try several common composition keys
    composition = (
        data.get('composition') or
        data.get('formula') or
        data.get('chemical_system') or
        data.get('elements') or
        data.get('surface_composition')
    )

    print(f"ID: {row[0]}")
    print(f"  γ = {gamma}")
    print(f"  Composition: {composition}\n")

conn.close()


ID: 5
  γ = -0.0396222932128175
  Composition: {"Zr": 30, "Os": 18}

ID: 6
  γ = -0.03910053176608415
  Composition: {"Zr": 30, "Os": 18}

ID: 7
  γ = -0.037023192676041004
  Composition: {"Zr": 30, "Os": 18}

ID: 8
  γ = -0.031646629501449745
  Composition: {"Zr": 30, "Os": 18}

ID: 23
  γ = -0.012784467136547571
  Composition: {"Zn": 15, "Au": 12, "Ag": 6}



In [3]:
# Verify by reading a few entries from the new database
conn = sqlite3.connect('filtered_surfaces.db')
cur = conn.cursor()

cur.execute("SELECT id, key_value_pairs FROM systems LIMIT 5")
for row in cur.fetchall():
    kvp = row[1]
    if isinstance(kvp, bytes):
        kvp = kvp.decode('utf-8')
    data = json.loads(kvp)
    print(f"ID: {row[0]}, γ = {data.get('surface_formation_energy_ev_per_ang2')}")
    
conn.close()


ID: 5, γ = -0.0396222932128175
ID: 6, γ = -0.03910053176608415
ID: 7, γ = -0.037023192676041004
ID: 8, γ = -0.031646629501449745
ID: 23, γ = -0.012784467136547571
